# GDA - Übung: Density-based Clustering zur ortsbezogenen Analyse von GPS Trajektorien

## Ziel der Übung
Das Ziel dieser Übung ist die Entwicklung/Anpassung eines Python Scripts, dass es Ihnen
erlaubt Haltepunkte aus Taxitrajektorien, die in Rom aufgezeichnet wurden, zu
extrahieren. Dazu sollen sie den DBSCAN (Density-based clustering) Algorithmus von
Ester, Kriegel, Sander und Xu (siehe Vorlesung) benutzen, um Bewegungsmuster in den
GPS Punkten durch deren Dichte in Raum und Zeit zu clustern.

## Zum GPS Trajektorien Datensatz
Die Daten stammen aus einem öffentlich verfügbaren Datensatz bei dem 320 Taxis mit
GPS Empfängern ausgestattet wurden und über einen Zeitraum von 30 Tagen getracked
wurden. Beim Tracking wird durchschnittlich 1 GPS Punkt alle 7 Sekunden erzeugt.
Dieser Datensatz wurde ursprünglich zur Verbesserung der Seuchenprävention
verwendet, wir werden ihn heute verwenden um die Haltepunkte der Taxis automatisch
zu identifizieren.

## Vorgehen

### 1) Vorüberlegung: Wie kann DBScan Haltepunkte erkennen
Rufen Sie sich ihr Wissen zu DBSCAN aus der Vorlesung in Erinnerung, überlegen Sie sich
wie Sie einen Haltepunkt für Taxis definieren können und welche raumzeitlichen
Anforderungen er erfüllen muss. Überlegen Sie sich Antworten auf folgende Fragen:
- Wie könnte ein sinnvoller Haltepunkt für Taxis definiert sein?
- Wie können Sie diese raumzeitlichen Anforderungen mit DBSCAN erfüllen?
- Wie beeinflussen die raumzeitlichen Anforderungen die Parameter von DBSCAN (min samples, eps, distance metric)?
- Was könnten gute Parameter sein?
Diskutieren Sie Ihre Antworten mit Ihrem Nachbarn/ Ihrer Nachbarin und notieren Sie
mögliche Lösungen.

### 2) Übersicht über Funktionen und das Programm bekommen
Hier eine kurze Übersicht über die Funktionen die in
der main Funktion aufgerufen werden. Die Methoden transform() und clustering_with_dbscan() müssen verändert werden.
- read_data(): Die Daten werden aus der .csv Datei in eine Liste [X, Y, T] (xkoordinate, y-koordinate, Zeitstempel) eingelesen.
- transform(): Reprojeziert die Daten.
- plot_nb_dists(): Erstellt das n-nearest-neighbours Diagramm.
- clustering_with_dbscan(): Berechnet die Cluster mit DBSCAN.
- plot_cluster(): Erstellt eine Grafik um Ihnen schnell das Ergebnis zu zeigen.
- export_to_shp(): Exportiert die GPS Punkte mit Ihrer Clusternummer als Shapefile.

In [ ]:
import datetime
import pyproj
import numpy as np


from gda_sdm_functions import read_data, plot_nb_dists, plot_cluster, export_to_shp


In [ ]:
# Path to geodatabase
input_file = r"taxi_21.txt"


# source projection
proj_wgs84 = pyproj.Proj("+init=EPSG:4326")

# target protection
proj_target = pyproj.Proj("+init=EPSG:25833")


### 3) Einlesen und Transformieren der Daten

#### Ziel: Verändern Sie die Funktion transform(), sodass die Koordinaten in ein geeignetes metrisches System projiziert werden und der Zeitstempel in kontinuierliche und interpretierbare Zeitwerte umgewandelt wird.
Die Funktion transform() ist noch unvollständig. Sie müssen die Funktion in einer Weise
implementieren, dass die Koordinaten interpretierbare räumliche Distanzen vorliegen (z.B.
Meter), und die Zeitwerte kontinuierliche und interpretierbare Zeitwerte (z.B. Sekunden)
sind. Beides brauchen Sie zur Festlegung des Distanzparameters im clustering.
Hinweis: wenn zwei datetime.datetime Objekte subtrahiert werden, wird ein
datetime.timedelta Objekt zurückgegeben, kein kontinuierlicher Wert.

Mittels pyproj können sie lon / lat Koordinaten in eine andere Projektion transformieren:

x, y = pyproj.transform(pyproj.Proj("+init=EPSG:4326"),
pyproj.Proj("+init=EPSG:<<target epsg code>>"),
lon,
lat)

In [ ]:
def transform(data_in):

    data_out = []

    t_reference = datetime.datetime(2014, 1, 1)
    
    # iterate over every point in the input data
    for d in data_in:
        x = d[0]
        y = d[1]
        ts = d[2]

        ts = 0
        
        data_out.append([x, y, ts])
    
    return data_out


### 4) Implementierung von DBSCAN
#### Ziel: Verändern Sie die Funktion clustering_with_dbscan(), sodass Sie die cluster labels
und die Indizies der core samples berechnet und ausgibt.
- Importieren Sie den DBSCAN Algorithmus aus Scikit-learn
- Initialisieren Sie ein DBSCAN Objekt (Übergeben Sie dabei die Input Daten)
- Nutzen Sie die fit() Methode des DBSCAN Objekts um ein Clustering zu berechnen
- Weisen Sie das Ergebnis des Clusterings den Variablen labels und core_samples_indices zu.

Hilfestellung:
Offizielle Dokumentation:
https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html
Beispiel:
https://scikit-learn.org/stable/auto_examples/cluster/plot_dbscan.html#sphx-glr-autoexamples-
cluster-plot-dbscan-py

In [ ]:
def clustering_with_dbscan(X, eps=1, min_samples=1, metric='cityblock'):
    """ Function derived from scipy dbscan example
    http://scikit-learn.org/stable/auto_examples/cluster/plot_dbscan.html#example-cluster-plot-dbscan-py
    """

    X = np.array(data)
    labels_placeholder = np.ones(len(data))


    ##########################################################################
    # Compute DBSCAN
    
    #Part1: Compute DBSCAN here---

    
    # assign labels and core_sample_indices to labels
    labels = np.random.randint(0, 5, size=len(data))
    core_samples_indices = [1,]
    
    #######################################################################
    

    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

    print('Estimated number of clusters: %d' % n_clusters_)

    return labels, core_samples_indices


In [ ]:
if __name__ == '__main__':

    # Read data, limit number of rows for speed
    org_data = read_data(input_file, nrows=2000)

    # Apply transformations
    data = transform(org_data)

    metric = 'euclidean'
    
    # plot nearest neighbor distances diagram
    plot_nb_dists(data, nearest_neighbor=[7,10,15], metric=metric, ylim=250)
    
    #set parameters
    eps = 1
    min_samples = 1
    
    # Calculate clusters with dbscan
    labels, core_samples_indices = clustering_with_dbscan(data, eps=eps, min_samples=min_samples, metric=metric)
    
    # plot clusters
    plot_cluster(data, labels, core_samples_indices, proj_wgs84=proj_wgs84, proj_target=proj_target, linestyle='solid')

    
    # export clusters to geodatabase
#    export_layer_name = "stops"
#    export_to_shp(data, labels, export_layer_name, crs=proj_target.srs)


### 5) Wahl der Parameter
Der Code und das Clustering sind jetzt voll Funktionsfähig. Das Ergebnis hängt allerdings
stark von den gewählten Parametern ab. Ziel ist nun, die Paramter so anzupassen dass plausible cluster generiert werden. Nähere Informationen finden sie auf dem Angabenblatt.
    
### 6) Ergebnisse Visualisieren und Parameter Tunen
Passen sie die Parameter der clustering_with_dbscan() Funktion (eps, min_samples, metric)
an und führen Sie sie aus. Plotten sie die gefunden Cluster mit der Funktion plot_cluster().
Schauen sie sich den Konsolen output und die Cluster im Plot an. Das Ergebnis der
Methode sind die Clusterlabels für jeden Punkt des Trajektoriendatensatzes. Wie liest man
aus dem Clusteringergebnis Clusterlabels und core points? Lassen sie sich das Ergebnis in der Konsole ausgeben und schauen Sie sich die cluster labels an. Ein Label von -1
bedeutet „Noise“.
    
### 7) Ergebnis exportieren und auf Karte visualisieren
Exportieren sie die gefunden Cluster mittels export_to_shp() als shapefile. Sehen Sie sich
das Clusterergebnis als einen Layer in ArcGIS/QGIS mit der Hintergrundkarte an (färben
Sie die Punkte nach Clusterzugehörigkeit ein).